# TP 2 - Organización de Datos 2019
##### Grupo 36 (oyentes)

Análisis de Supervivencia: Me permite determinar el tiempo que lleva que ocurra determinado evento (cita)

En el presente trabajo se pretende calcular:
* En un instante dado, estimar 𝑆𝑡(𝑑) el tiempo hasta que un dispositivo 𝑑 aparezca de vuelta en una subasta RTB
* En un instante dado, estimar 𝑆𝑐(𝑑) el tiempo hasta que un dispositivo 𝑑 convierta


●	Descomprimir por fuera de pandas los archivos
○	Desde python se pueden usar estos snippets para descomprimir sin ocupar toda la memoria: https://gist.github.com/CrossNox/47675490e43a3ef305a13ca56750e045
●	Los datos de entrenamiento van del 18/04 al 26/04 inclusive
●	Los datos de objetivo se armaron con datos del 28/04 al 30/04 inclusive
●	Es decir, el maximo valor sera de 3 dias
●	Con los datos de entrenamiento se pueden entrenar/validar 5 ventanas  de 3 dias
○	Del 18 al 20
○	Del 19 al 21
○	Del 20 al 22
○	Del 21 al 23
○	Del 22 al 24
■	Cada una se valida contra los siguientes tres días


In [67]:
#Leo el archivo INSTALLS
import pandas as pd
from datetime import datetime

installs = pd.read_csv("F:/Materias/Organizacion_d_datos_2019/TP2/installs.csv.gzip",
                       compression = 'gzip', dtype='object')

#Explicito el formato de la fecha
installs['created'] = installs['created'].astype('datetime64[ns]')

installs['created'] = (installs['created'].datetime(2019,3,9)).total_seconds()

installs.head()

AttributeError: 'Series' object has no attribute 'datetime'

In [52]:
# http://benalexkeen.com/decision-tree-classifier-in-python-using-scikit-learn/

installs = installs[['ref_hash', 'device_model', 'device_countrycode', 'implicit','attributed']]

installs.head()

,ref_hash,device_model,device_countrycode,implicit,attributed
0,4716708407362582887,3.739127126472163e+17,6287817205707153877,True,False
1,7143568733100935872,7.80553892759877e+18,6287817205707153877,False,False
2,5230323462636548010,8.355495513718673e+18,6287817205707153877,True,False
3,5097163995161606833,2.3557720913769155e+18,6287817205707153877,True,False
4,6328027616411983332,6.156971151807135e+18,6287817205707153877,False,False


In [53]:
# Si hubiese datos categoricos deberia ocuparme acá

installs['attributed'] = installs['attributed'].map({'False': 0, 'True': 1})
installs['implicit'] = installs['implicit'].map({'False': 0, 'True': 1})

installs = installs.dropna()

installs.head()

,ref_hash,device_model,device_countrycode,implicit,attributed
0,4716708407362582887,3.739127126472163e+17,6287817205707153877,1,0
1,7143568733100935872,7.80553892759877e+18,6287817205707153877,0,0
2,5230323462636548010,8.355495513718673e+18,6287817205707153877,1,0
3,5097163995161606833,2.3557720913769155e+18,6287817205707153877,1,0
4,6328027616411983332,6.156971151807135e+18,6287817205707153877,0,0


In [54]:
# preparo ejes

X = installs.drop('implicit', axis=1)
y = installs['implicit']

y.head()

0    1
1    0
2    1
3    1
4    0
Name: implicit, dtype: int64

In [55]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [56]:
from sklearn import tree

model = tree.DecisionTreeClassifier()
model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [57]:
# fittear el modelo a los datos de entrenamiento

model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [58]:
#Then we score the predicted output from model on our test data against our ground truth test data.

y_predict = model.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)

0.6500373938674058

In [59]:
#Let’s also take a look at our confusion matrix:

from sklearn.metrics import confusion_matrix

pd.DataFrame(
    confusion_matrix(y_test, y_predict),
    columns=['Predicted Not Survival', 'Predicted Survival'],
    index=['True Not Survival', 'True Survival']
)

,Predicted Not Survival,Predicted Survival
True Not Survival,71515,18877
True Survival,20898,2365


################ LINEA

In [ ]:
from lifelines import KaplanMeierFitter


event_observed = [True, True, False, True, True, True, True, False, False, True, True,
                  True, True, True, True, True, False, True, False, True]

kmf = KaplanMeierFitter()
kmf.fit(durations, event_observed)
kmf.plot()

In [ ]:
# https://lifelines.readthedocs.io/en/latest/Survival%20Analysis%20intro.html

from lifelines.plotting import plot_lifetimes
import numpy as np
from numpy.random import uniform, exponential

N = 25

CURRENT_TIME = 10

actual_lifetimes = np.array([
    exponential(12) if (uniform() < 0.5) else exponential(2) for i in range(N)
])
observed_lifetimes = np.minimum(actual_lifetimes, CURRENT_TIME)
death_observed = actual_lifetimes < CURRENT_TIME

ax = plot_lifetimes(observed_lifetimes, event_observed=death_observed)

ax.set_xlim(0, 25)
ax.vlines(10, 0, 30, lw=2, linestyles='--')
ax.set_xlabel("time")
ax.set_title("Births and deaths of our population, at $t=10$")
print("Observed lifetimes at time %d:\n" % (CURRENT_TIME), observed_lifetimes)